#### **Check GPU availability**

In [ ]:
import tensorflow as tf

# Check if GPU is available
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

# Display GPU name
!nvidia-smi


#### **Configure Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Change directory to your dataset
import os
os.chdir('/content/drive/MyDrive/Severstal')  # Change this path


#### **Install Dependencies**

In [ ]:
!pip install tensorflow keras opencv-python matplotlib numpy albumentations

#### **Train Test Split**

In [ ]:
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split

# Load CSV file
df = pd.read_csv('/content/drive/MyDrive/Severstal/train.csv')

# Drop NaN (if any images do not have segmentation masks)
df = df.dropna(subset=['EncodedPixels'])

# Unique Image IDs
image_ids = df["ImageId"].unique()

# Train-Test-Validate Split (70-20-10)
train_ids, temp_ids = train_test_split(image_ids, test_size=0.30, random_state=42)
val_ids, test_ids = train_test_split(temp_ids, test_size=0.66, random_state=42)  # 20% Test, 10% Validation

# Define directories
base_dir = "/content/drive/MyDrive/Severstal/train_images/"
output_dir = "/content/dataset/"

for folder in ["train", "test", "validate"]:
    os.makedirs(os.path.join(output_dir, folder), exist_ok=True)

# Function to move files
def move_files(image_list, dest_folder):
    for img in image_list:
        src_path = os.path.join(base_dir, img)
        dest_path = os.path.join(output_dir, dest_folder, img)
        shutil.copy(src_path, dest_path)

# Move images to respective folders
move_files(train_ids, "train")
move_files(test_ids, "test")
move_files(val_ids, "validate")

print(f"✅ Dataset split completed! Train: {len(train_ids)}, Test: {len(test_ids)}, Validate: {len(val_ids)}")


#### **U-Net Model Base**

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model

# Enable Mixed Precision for Faster Training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

# U-Net Model with EfficientNet Backbone
def build_unet(input_shape=(256, 256, 3)):
    inputs = tf.keras.Input(input_shape)

    # EfficientNet as Backbone (Feature Extractor)
    backbone = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")
    backbone.trainable = False  # Freeze backbone during initial training

    # U-Net Decoder
    conv1 = Conv2D(256, (3, 3), activation="relu", padding="same")(backbone.output)
    up1 = UpSampling2D((2, 2))(conv1)
    
    conv2 = Conv2D(128, (3, 3), activation="relu", padding="same")(up1)
    up2 = UpSampling2D((2, 2))(conv2)

    conv3 = Conv2D(64, (3, 3), activation="relu", padding="same")(up2)
    up3 = UpSampling2D((2, 2))(conv3)

    conv4 = Conv2D(32, (3, 3), activation="relu", padding="same")(up3)
    up4 = UpSampling2D((2, 2))(conv4)

    outputs = Conv2D(1, (1, 1), activation="sigmoid")(up4)

    return Model(inputs, outputs)

# Compile Model
model = build_unet()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()


#### **Train the Model**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image Generators for Data Augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=15, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load Images in Batches
train_generator = train_datagen.flow_from_directory(
    "/content/dataset/train/",
    target_size=(256, 256),
    batch_size=32,
    class_mode="binary"
)

val_generator = val_datagen.flow_from_directory(
    "/content/dataset/validate/",
    target_size=(256, 256),
    batch_size=32,
    class_mode="binary"
)

# Train Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

# Save Model
model.save("/content/drive/MyDrive/Severstal/Collab/steel_defect_unet.h5")
print("✅ Model training complete & saved!")


#### **Inference**

import matplotlib.pyplot as plt
import numpy as np

# Load Model
model = tf.keras.models.load_model("/content/drive/MyDrive/Severstal/Collab/steel_defect_unet.h5")

# Load Test Image
test_img_path = "/content/dataset/test/sample.jpg"
test_img = tf.keras.preprocessing.image.load_img(test_img_path, target_size=(256, 256))
test_img = np.array(test_img) / 255.0  # Normalize
test_img = np.expand_dims(test_img, axis=0)

# Predict Mask
pred_mask = model.predict(test_img)[0]
pred_mask = np.squeeze(pred_mask)  # Remove extra dimensions

# Show Results
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.imshow(np.squeeze(test_img), cmap="gray")
plt.title("Original Image")

plt.subplot(1, 2, 2)
plt.imshow(pred_mask, cmap="jet", alpha=0.5)
plt.title("Predicted Mask")

plt.show()
